In [12]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from py4j.java_gateway import java_import
#%pip install delta-spark
from delta.tables import *   
#to create delta dataframe in particular location we need this library 

spark=SparkSession.builder.getOrCreate()

schema=StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("city", StringType(), True),
    StructField("country", StringType(), True),
    StructField("contact_no", IntegerType(), True)
])

data=[(1000, "John", "NYC", "USA", 9846472)]
#'df' is our source dataframe
df=spark.createDataFrame(data, schema)
display(df)


#now we will insert few records, one would have same emp_id as earlier record but we are chaning the 'name' column and other record will be new.
data=[(1000, "Steve", "NYC", "USA", 9846472), (2000, "Peter", "London", "UK", 1003833)]
df=spark.createDataFrame(data, schema)

#delta_df is the target dataframe
delta_df=DeltaTable.forPath(spark, "/pyspark_practice/delta_merge")

#below is merge operation between both source and target dataframes (df and delta_df) logic: 
delta_df.alias("target").merge(
    source=df.alias("source"),
    condition="target.emp_id==source.emp_id"    #here we are checking whether emp_id of df == emp_id delta_df
).whenMatchedUpdate(set={
    "name":"source.name",
    "city":"source.id",
    "country":"source.country",
    "contact_no":"source.contact_no"     #If emp_id of df == emp_id delta_df, then set new values to the existing emp_id
}).whenNotMatchedInsert(values={
    "emp_id":"source.emp_id",
    "name":"source.name",
    "city":"source.id",
    "country":"source.country",
    "contact_no":"source.contact_no"   #If its new record, then insert new records in delta_df
}).execute()


display(delta_df)

#The main difference between Join() and merge() is : 
#In join(), we are just selecting/retrieving based on the provided condition. 
#But In merge(), we can perform updation of existing records based on the common columns between 2 dataframes or we can insert new records

DataFrame[emp_id: int, name: string, city: string, country: string, contact_no: int]

TypeError: 'JavaPackage' object is not callable